## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/home/faifai/anaconda3/envs/deeplearning-gpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Convolution2D(64, (3, 3), activation='relu'))
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(128, (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(rate=0.5)) ##Dropout

#輸出
classifier.add(Dense(units=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=128,epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
50000/50000 [==============================] - 8s 157us/step - loss: 1.6701 - accuracy: 0.5097
Epoch 2/100
50000/50000 [==============================] - 7s 136us/step - loss: 1.1487 - accuracy: 0.6692
Epoch 3/100
50000/50000 [==============================] - 7s 140us/step - loss: 0.9768 - accuracy: 0.7230
Epoch 4/100
50000/50000 [==============================] - 7s 138us/step - loss: 0.8927 - accuracy: 0.7534
Epoch 5/100
50000/50000 [==============================] - 7s 145us/step - loss: 0.8363 - accuracy: 0.7792
Epoch 6/100
50000/50000 [==============================] - 7s 138us/step - loss: 0.7951 - accuracy: 0.7971
Epoch 7/100
50000/50000 [==============================] - 7s 141us/step - loss: 0.7584 - accuracy: 0.8187
Epoch 8/100
50000/50000 [==============================] - 7s 143us/step - loss: 0.7309 - accuracy: 0.8299
Epoch 9/100
50000/50000 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[8.2542337e-03, 4.6718037e-06, 2.6595894e-02, 3.0555191e-02,
        4.1843432e-01, 7.7018968e-04, 4.9479073e-01, 4.1275754e-05,
        2.0369759e-02, 1.8369491e-04]], dtype=float32)